In [3]:
company_summary = '''
이 회사는 기업을 위한 SaaS(Software as a Service) 솔루션을 제공하는 IT 기업으로, 주로 경영 지원 시스템의 비효율성을 해결하는 데 중점을 두고 있습니다. 회사의 주요 제품과 서비스는 다음과 같습니다:

### 문제점
1. **비효율적인 경영 지원 시스템**: 많은 기업들이 여전히 수작업으로 반복적인 업무를 처리하고 있으며, 이는 전체 업무 시간의 40% 이상을 차지합니다.
2. **커뮤니케이션 오버헤드**: 팀 간의 비효율적인 커뮤니케이션으로 인해 추가적인 시간과 비용이 발생합니다.
3. **자산 관리의 어려움**: 유형 자산 및 무형 자산의 관리가 비효율적으로 이루어지고 있습니다.

### 솔루션
1. **SaaS 기반 경영 지원 시스템**: 
   - **자동화**: 반복적인 업무를 자동화하여 효율성을 높입니다.
   - **AI 에이전트**: AI 기반의 에이전트를 통해 실시간으로 데이터를 분석하고 인사이트를 제공합니다.
   - **자산 관리**: QR 코드 기반의 자산 관리 시스템을 통해 자산의 효율적인 관리가 가능합니다.

2. **커뮤니케이션 효율화**:
   - **통합 커뮤니케이션 플랫폼**: 다양한 커뮤니케이션 도구를 통합하여 오버헤드를 줄입니다.
   - **데이터 기반 인사이트**: 커뮤니케이션 데이터를 분석하여 효율성을 높이는 전략을 제안합니다.

3. **데이터 인사이트 제공**:
   - **실시간 데이터 분석**: 기업 데이터를 실시간으로 분석하여 경영 인사이트를 제공합니다.
   - **리스크 및 유효 기간 관리**: 자산의 유효 기간을 관리하고 만료 알림을 제공합니다.

### 비즈니스 모델
- **서비스 제공**: 국내 기업에 SaaS 및 솔루션을 제공하며, 임직원 서비스도 함께 제공합니다.
- **수익 모델**: 월 사용료 및 라이선스 비용을 통해 수익을 창출합니다. 다양한 플랜(베이직, 비즈니스, 엔터프라이즈)을 통해 고객의 필요에 맞춘 서비스를 제공합니다.

### 시장 및 목표
- **시장 확장**: 글로벌 빅데이터 및 분석 솔루션 시장으로의 확장을 목표로 하고 있습니다.
- **매출 목표**: 2030년까지 매출 11000억 원 달성을 목표로 하고 있습니다.

### 팀 구성
- **CEO**: 유민재
- **CTO**: 김종민
- **개발자**: 이승현

이 회사는 IT 자산 관리 및 데이터 분석 솔루션을 통해 기업의 경영 효율성을 높이고, 글로벌 시장으로의 확장을 목표로 하고 있습니다.
'''

In [ ]:
import os
import json
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,
    api_key=OPENAI_API_KEY,
    model_kwargs={"response_format": {"type": "json_object"}}
)

def generate_lead_candidates(company_summary, llm):
    prompt = f"""
    당신은 B2B 세일즈 전략 전문가입니다.

    아래는 우리 회사의 요약 정보입니다. 이 정보를 바탕으로,
    **우리 솔루션이 필요할 가능성이 높은 국내 기업 10곳**을 찾아주세요.

    우리 회사 정보:
    {company_summary}

    조건:
    - 실제로 존재하는 국내 기업일 것
    - 경영지원 시스템, AI 자동화, 자산관리, 데이터 기반 인사이트 제공이 필요한 기업
    - 경쟁사가 아닌, 잠재 고객일 것
    - 각 리드에는 회사명, 산업군, 추정 매출액, 펀딩 금액, 홈페이지, 대표자명, 주소, 이메일, 전화번호, 관련성 점수 및 이유를 포함
    - 정보가 없는 항목은 비워 두어도 됨.
    - 국가는 대한민국
    - 지역은 파트너 위치와 가까울수록 선호
    - 산업군은 NCS 기준으로 경쟁업체이면 배제
    - 직원수는 많을수록 선호
    - 매출은 많을수록 선호
    - 순이익은 많을수록 선호
    - 당면 문제는 파트너가 해결하는 문제와 유사할수록 선호
    
    조건 2:
    - 국내 중소~중견기업 (직원 수 100~500명, 비상장 등)
    - 우리 회사 정보를 보고 판단할 때 서비스를 필요로 하는 기업
    - 다음 대기업은 절대 포함하지 말 것: 삼성, LG, KT, SK, 카카오, 네이버, 현대 등
    - 스타트업이라면 Series A~C 단계로 성장 가능성 있는 곳
    - 반드시 실제로 존재하는 기업명을 쓸 것, 가상의 회사나 없는 정보는 절대 쓰지 말 것
    - 각 리드에 대해 관련성 점수를 0.0에서 1.0 사이로 평가하고, 그 이유를 설명할 것

    반드시 아래 JSON 형식으로만 응답하세요:

    {{
        "leads": [
        {{
        "company": "회사명",
        "industry": "산업",
        "sales": 매출액(숫자),
        "total_funding": 총 펀딩 금액(숫자),
        "homepage": "웹사이트 URL", 
        "key_executive": "주요 경영진",
        "address": "회사 주소",
        "email": "연락 이메일",
        "phone_number": "연락처",
        "relevance_score": 0.0~1.0 사이의 관련성 점수",
        "reasoning": "이 회사가 우리에게 적합한 영업 타겟인 이유를 설명 (산업 시너지, 문제 해결 가능성 등)"
        }},
        ...
    ]
    }}
    """

    messages = [{"role": "user", "content": prompt}]
    response = llm.invoke(messages)
    return json.loads(response.content)

In [9]:
lead_data = generate_lead_candidates(company_summary, llm)

# pandas로 정리
import pandas as pd
df_leads = pd.DataFrame(lead_data["leads"])
df_leads.to_csv("step1_LLM_생성리드.csv", index=False)

# 확인
print(df_leads.head())

   company       industry          sales  total_funding  \
0    더존비즈온  소프트웨어 개발 및 공급  1200000000000              0   
1   한글과컴퓨터       소프트웨어 개발   350000000000              0   
2  더존ICT그룹   IT 서비스 및 솔루션   500000000000              0   
3    비즈플레이     경영지원 소프트웨어    15000000000    20000000000   
4      윌비스    교육 및 IT 서비스    80000000000              0   

                    homepage key_executive                address  \
0    https://www.duzon.co.kr      김용우 대표이사    경기도 성남시 분당구 판교로 242   
1     https://www.hancom.com      김상철 대표이사  경기도 성남시 분당구 대왕판교로 644   
2   https://www.duzonict.com      이용우 대표이사    경기도 성남시 분당구 판교로 242   
3  https://www.bizplay.co.kr      김태훈 대표이사     서울특별시 강남구 테헤란로 311   
4    https://www.willbes.net      김동현 대표이사      서울특별시 강남구 논현로 508   

                   email  phone_number  relevance_score  \
0    contact@duzon.co.kr  031-620-3000             0.85   
1        info@hancom.com  031-724-3000             0.80   
2   contact@duzonict.com  031-620-4000             0.

## company_data에서 전화번호 추출

In [31]:
import pandas as pd
import re

# 1. 회사명 정규화 함수
def normalize_company_name(name: str) -> str:
    if pd.isna(name):
        return ""
    return (
        str(name)
        .replace("(", "")
        .replace(")", "")
        .replace("㈜", "")
        .replace("주식회사", "")
        .replace(",", "")
        .replace(".", "")
        .replace(" ", "")
        .strip()
    )

# 2. 유효한 전화번호인지 확인
def is_valid_phone(phone):
    if pd.isna(phone):
        return False
    phone = str(phone)
    return bool(re.match(r"\d{2,4}-\d{3,4}-\d{4}", phone))

# 3. 전화번호 보강 메인 함수
def enrich_phone_numbers_safely(df_leads, company_db):
    df_leads = df_leads.copy()

    # 회사명 정규화
    df_leads["company_normalized"] = df_leads["company"].apply(normalize_company_name)
    company_db["company_normalized"] = company_db["company"].apply(normalize_company_name)

    # 유효한 전화번호만 필터링
    company_db = company_db[company_db["phone_number"].apply(is_valid_phone)]

    # 병합 (정규화된 회사명 기준)
    df_merged = pd.merge(
        df_leads,
        company_db[["company_normalized", "phone_number"]],
        on="company_normalized",
        how="left",
        suffixes=("", "_from_db")
    )

    # DB 전화번호로 덮어쓰기 (combine_first는 왼쪽 값 우선)
    df_merged["phone_number"] = df_merged["phone_number_from_db"].combine_first(df_merged["phone_number"])

    # 정리
    df_merged.drop(columns=["company_normalized", "phone_number_from_db"], inplace=True)

    return df_merged

In [ ]:
df_leads = pd.DataFrame(lead_data["leads"])

# 회사 DB 로드
company_db = pd.read_csv("company_data.csv")

# 전화번호 보강
df_leads_updated = enrich_phone_numbers_safely(df_leads, company_db)

# 확인
print(df_leads_updated.head(10))

   company          industry          sales  total_funding  \
0    더존비즈온     소프트웨어 개발 및 공급  1200000000000              0   
1   한글과컴퓨터          소프트웨어 개발   350000000000              0   
2  더존ICT그룹      IT 서비스 및 솔루션   500000000000              0   
3    비즈플레이        경영지원 소프트웨어    15000000000    20000000000   
4      윌비스       교육 및 IT 서비스    80000000000              0   
5    에스에스알      IT 서비스 및 솔루션   120000000000              0   
6     인크로스  디지털 마케팅 및 IT 솔루션    70000000000              0   
7     알서포트     원격지원 및 IT 솔루션    90000000000              0   
8     알서포트     원격지원 및 IT 솔루션    90000000000              0   
9     케이티스      IT 서비스 및 솔루션   110000000000              0   

                    homepage key_executive                address  \
0    https://www.duzon.co.kr      김용우 대표이사    경기도 성남시 분당구 판교로 242   
1     https://www.hancom.com      김상철 대표이사  경기도 성남시 분당구 대왕판교로 644   
2   https://www.duzonict.com      이용우 대표이사    경기도 성남시 분당구 판교로 242   
3  https://www.bizplay.co.kr      김태훈 대표이